# Presentación

👋 Este cuaderno colab es un recurso generado por Zoraida Callejas y David Griol para la asignatura de [Tecnologías del habla y del Lenguaje Natural](https://masteres.ugr.es/desarrollo-software/docencia/plan-estudios/guia-docente/M52/56/5/23) del Máster de Desarrollo de Software de la Universidad de Granada. La primera parte del cuaderno es una adaptación de la unidad 6 del curso Hugging Face Audio compartido [aquí](https://huggingface.co/learn/audio-course/) y la descipción del modelo MMS de Facebook para español compartido [aquí](https://huggingface.co/facebook/mms-tts-spa). Dos recursos que te recomendamos investigar.

Siguiendo el cuaderno aprenderás a:

*   Utilizar **modelos pre-entrenados** basados en *deep learning* para la síntesis del habla.
*   Guardar los audios generados en archivos wav.
* Guardar archivos en tu Google Drive.

Puedes ir siguiéndolo y ejecutando cada celda de código secuencialmente.

# Instalaciones

Comienza instalando los elementos que necesitarás. Seguramente algunos ya los tengas (en ese caso se mostrará "Requirement already satisfied").

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install scipy

# Descarga del modelo TTS pre-entretenado

Se propone comenzar con el modelo TTS **SpeechT5 de Microsoft** disponible en [HuggingFace](https://huggingface.co/docs/transformers/main/en/model_doc/speecht5). Se trata de un modelo de 2022, que vamos a usar en inglés. Puedes leer más sobre el modelo [aquí](https://github.com/microsoft/SpeechT5?tab=readme-ov-file).

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

tokenizadorT5 = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
modeloT5 = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

# Tokeniza



Vamos a tokenizar el texto que queremos sintetizar. Lo escribimos en inglés porque desafortunadamente este modelo solo está disponible en ese idioma.




In [ ]:
texto = "I love learning about speech synthesis"
token_ids = tokenizadorT5(text=texto)

#Imprimimos el valor numérico que se ha asociado a los tokens:
print("Ids de los tokens", token_ids.input_ids)

#Imprimimos ahora el token original
raw_tokens = [tokenizadorT5.decode([token_id]) for token_id in token_ids.input_ids]
print("Raw tokens:", raw_tokens)

Al tokenizar puedes pedir que se generen tensores. Los tensores son objetos matemáticos que almacenan valores numéricos. Un tensor de dimensión 0 es un valor, de dimesnion 1 un vector, de dimension 2 una matriz... pudiendo tener más de 3 dimensiones. Representan los datos de entrada, pesos, sesgos y valores de salida durante los procesos de entrenamiento e inferencia con redes neuronales.

Distintos frameworks para deep learning generan distintos tipos de tensores. Usualmente es posible elegir entre 4:
*   'pt' - Pytorch <-- Este es el que vamos utilizar
*   'tf' - TensorFlow
*   'np' - NumPy
*   'jax' - JAX

In [ ]:
import torch

token_ids = tokenizadorT5(text=texto, return_tensors="pt")
input_ids = token_ids.input_ids
print(input_ids)

# Utiliza speaker embeddings



Con los speaker embeddings, puedes indicar qué voz o tipo de voz quieres para tu sintentizador. Se propone utilizar los embeddings compartidos [aquí](https://huggingface.co/datasets/Matthijs/cmu-arctic-xvectors) correspondientes a las grabaciones de una base de datos de voces abierta [compartida por la Universidad Carnegie Mellon](http://www.festvox.org/cmu_arctic/).

In [ ]:
from datasets import load_dataset

# Cargamos el dataset, que está dividido en varios subconjuntos o "splits", utilizaremos el split de "validation"
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

#Vamos a utilizar los embeddings de la voz 7306
speaker_embeddings = embeddings_dataset[7306]["xvector"]

#Lo convertimos en un tensor PyTorch
speaker_embeddings = torch.tensor(speaker_embeddings).unsqueeze(0)

print(speaker_embeddings)

# Genera el audio

¡Llegó el momento de generar el audio! 🙌 Para ello vas a usar los tensores y speaker embeddings que has generado en los pasos previos y además un vocoder. En este caso, se propone el vocoder [HiFi-GAN](https://huggingface.co/microsoft/speecht5_hifigan) de Microsoft.



In [ ]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
habla = modeloT5.generate_speech(token_ids["input_ids"], speaker_embeddings, vocoder=vocoder)

# Escucha el resultado 🤞

In [ ]:
from IPython.display import Audio

Audio(habla, rate=16000)

# Prueba un modelo que funcione en español

Vamos a repetir el proceso anterior con un modelo que admite español. Para ello, vamos a usar el modelo **MMS-TTS de Facebook** [publicado en 2023](https://arxiv.org/abs/2305.13516). Puedes consultar [aquí](https://dl.fbaipublicfiles.com/mms/misc/language_coverage_mms.html) el código correspondiente a los idiomas que soporta. Para este cuaderno puedes usar el modelo en español *mms-tts-spa*.

In [ ]:
from transformers import VitsModel, VitsTokenizer
import torch

modeloMMS = VitsModel.from_pretrained("facebook/mms-tts-spa")
tokenizerMMS = VitsTokenizer.from_pretrained("facebook/mms-tts-spa")

token_ids = tokenizerMMS("Me interesan las tecnologías del habla", return_tensors="pt")
input_ids = token_ids["input_ids"]


with torch.no_grad():
    outputs = modeloMMS(input_ids)

hablaES = outputs["waveform"]
Audio(hablaES, rate=16000)

# Guarda el audio en un archivo wav

En primer lugar, genera el archivo wav usando *scipy*.

In [ ]:
import scipy
import numpy as np

output_array = hablaES[0].numpy()
archivoAudio = "MiPrimeraVozSintetica.wav"
scipy.io.wavfile.write(archivoAudio, rate=modeloMMS.config.sampling_rate, data=output_array)

# Guarda el archivo wav en Google Drive

Te puede resultar interesante guardar el archivo en tu drive, a continuación aprenderás cómo. Al ejecutar este trozo de código, Google Colab pedirá permiso para acceder a tu directorio Drive, deberás conceder los permisos para que funcione. Si no quieres hacerlo no hay problema, en este cuaderno mandas tú.

In [ ]:
import shutil
from google.colab import drive

drive.mount('/content/drive')

origen = 'MiPrimeraVozSintetica.wav'
destino = '/content/drive/My Drive/MiPrimeraVozSintetica.wav'
shutil.move(origen, destino)